In [4]:
import os
from pathlib import Path
from dotenv import load_dotenv

## Loading Environment File

In [5]:
secret_path = Path("../.env")
print("Looking for dev.env at:", secret_path.resolve())
if secret_path.exists():
    load_dotenv(secret_path)
    print("Loaded environment variables from dev.env")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in environment variables or dev.env file.")
print("GOOGLE_API_KEY loaded:", bool(GOOGLE_API_KEY))

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OPENROUTER_API_KEY not found in environment variables or dev.env file.")
print("OPENROUTER_API_KEY loaded:", bool(OPENROUTER_API_KEY))

Looking for dev.env at: C:\Users\gabri\workspace\aida_projects\vaxtalk\.env
Loaded environment variables from dev.env
GOOGLE_API_KEY loaded: True
OPENROUTER_API_KEY loaded: True


## Setup: Imports, Retry Config

In [6]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


Add the project root to sys.path to easily import our classes:

In [4]:
import sys

# Ensure the project root (the parent of the "src" directory) is on sys.path
# so that "import src.model" finds the src package under the project root.
project_root = Path.cwd().parent
src_dir = project_root / "src"

project_root_path = str(project_root.resolve())
if project_root_path not in sys.path:
    sys.path.insert(0, project_root_path)

from src.model import Intensity, SentimentOutput
from src.model.rag_output import RagOutput

Create a general retry policy:

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Agents Creation

In [6]:
# RAG system
rag_agent = Agent(
    name="RAG_Vaccine_Informer",
    model=Gemini(
        model="gemini-2.5-flash-lite", 
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="rag_output",  # The result of this agent will be stored in the session state with this key.
    output_schema=RagOutput,  # Define the expected output schema
)

print("✅ RAG created.")

✅ RAG created.


In [7]:
# Health Researcher: Focuses on medical breakthroughs.
sentiment_agent = Agent(
    name="sentiment_analysis",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""XXX""",
    tools=[],
    output_key="sentiment_output",  # The result will be stored with this key.
    output_schema=SentimentOutput,  # Define the expected output schema.
)

print("✅ sentiment_agent created.")

✅ sentiment_agent created.


In [8]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these findings into a single executive summary:

    **RAG output:**
    {rag_output}
    
    **User sentiment analysis:**
    {sentiment_output}
""",
    output_key="final_output",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [9]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_rag_sentiment_agent = ParallelAgent(
    name="ParallelRAGAndSentimentTeam",
    sub_agents=[sentiment_agent, rag_agent],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_rag_sentiment_agent, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


## Engine Def

In [ ]:
APP_NAME = "VaxTalkAssistant"
SQL_ASYNC_DRIVER = "aiosqlite"
DB_URL = f"sqlite+{SQL_ASYNC_DRIVER}:///vaxtalk_sessions.db"  # Local SQLite file

In [12]:
# InMemorySessionService stores conversations in RAM (temporary)
session_service = InMemorySessionService()

# Persistent memory using a SQLite database
# SQLite database will be created automatically
session_service = DatabaseSessionService(db_url=DB_URL)

In [ ]:
runner = Runner(
    agent=root_agent, 
    app_name=APP_NAME, 
    session_service=session_service
    )

## Testing

In [11]:
response = await runner.run_debug(
    "Should I vaccinate?"
)


 ### Created new session: debug_session_id

User > Should I vaccinate?


  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\gabri\workspace\aida_projects\vaxtalk\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3697, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\gabri\AppData\Local\Temp\ipykernel_26908\1760036474.py", line 1, in <module>
  |     response = await runner.run_debug(
  |                ^^^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\gabri\workspace\aida_projects\vaxtalk\.venv\Lib\site-packages\google\adk\runners.py", line 1054, in run_debug
  |     async for event in self.run_async(
  |   File "c:\Users\gabri\workspace\aida_projects\vaxtalk\.venv\Lib\site-packages\google\adk\runners.py", line 454, in run_async
  |     async for event in agen:
  |   File "c:\Users\gabri\workspace\aida_projects\vaxtalk\.venv\Lib\site-packages\google\adk\runners.py", line 442, in _run_with_trace
  |     async for event in agen:
  |   File "c:\Users\gabri\workspace\aida_projects